In [ ]:
import collections
from typing import List, Tuple

# Define a simple "State" object to hold the (benefit, count) pair
# This lets us use the .benefit and .count syntax from your pseudocode
State = collections.namedtuple('State', ['benefit', 'count'])

def solve_knapsack_with_min_items(groups: List[List[Tuple[int, int]]], W: int) -> State:
    """
    Solves the Grouped Knapsack problem, prioritizing max benefit,
    then min item count.

    Args:
        groups: A list of groups. Each group is a list of items.
                Each item is a (cost, benefit) tuple.
        W: The maximum budget (total cost).

    Returns:
        The final State(benefit, count) for the optimal solution.
    """

    # m is the number of groups
    m = len(groups)

    # Initialize the DP table.
    # DP[k][w] will store the best State (benefit, count)
    # using the first k groups and a budget of w.
    # We use m+1 and W+1 for easier 1-based indexing (k=0 is base case).
    # The default state is (benefit=0, count=0)
    DP = [[State(0, 0) for _ in range(W + 1)] for _ in range(m + 1)]

    # Loop for k from 1 to m (inclusive)
    for k in range(1, m + 1):
        # Get the actual group items (k-1 for 0-based list indexing)
        current_group = groups[k - 1]

        # Loop for w from 0 to W (inclusive)
        for w in range(W + 1):

            # Start by assuming the best option is to *skip* group k.
            # This is the "best" from the previous state (k-1)
            # with the same budget (w).
            best = DP[k - 1][w]

            # Now, iterate through each item 'i' in the current group
            # to see if taking one of them is a better option.
            for item_cost, item_benefit in current_group:

                # Check if we can afford this item
                if item_cost <= w:
                    # 'cand' = candidate state if we *do* pick this item

                    # Get the best state from *before* this group (k-1)
                    # using the *remaining* budget (w - item_cost)
                    prev_state = DP[k - 1][w - item_cost]

                    candBenefit = prev_state.benefit + item_benefit
                    candCount = prev_state.count + 1

                    # --- This is the core comparison logic ---

                    # 1. If this item gives a strictly better benefit
                    if candBenefit > best.benefit:
                        best = State(candBenefit, candCount)

                    # 2. Else, if benefits are equal, check the tie-breaker
                    elif candBenefit == best.benefit and candCount < best.count:
                        best = State(candBenefit, candCount)

            # After checking all items (and the "skip" option),
            # store the winning 'best' state in our table.
            DP[k][w] = best

    # The final answer is in the bottom-right corner:
    # using all 'm' groups and the full budget 'W'.
    return DP[m][W]

# --- Example Usage ---

# groups[i] = list of (cost, benefit) tuples for group i

# Group 0:
# Item A: cost=5, benefit=10
# Item B: cost=4, benefit=8
group_0 = [(5, 10), (4, 8)]

# Group 1:
# Item C: cost=3, benefit=7
group_1 = [(3, 7)]

# Group 2:
# Item D: cost=2, benefit=5
# Item E: cost=3, benefit=9
group_2 = [(2, 5), (3, 9)]

all_groups = [group_0, group_1, group_2]
max_budget = 10

# --- Run the algorithm ---
final_solution = solve_knapsack_with_min_items(all_groups, max_budget)

print(f"All Groups: {all_groups}")
print(f"Max Budget: {max_budget}")
print("---")
print(f"Highest Benefit: {final_solution.benefit}")
print(f"Fewest Items:    {final_solution.count}")
print(f"Final State:     {final_solution}")

# Example 2: Budget = 8
# Optimal solution would be Item A (5, 10) + Item C (3, 7)
# -> Total Benefit = 17, Total Count = 2
# Another solution: Item E (3, 9) + Item C (3, 7)
# -> Total Benefit = 16, Total Count = 2
# Another solution: Item E (3, 9) + Item B (4, 8)
# -> Total Benefit = 17, Total Count = 2
final_solution_8 = solve_knapsack_with_min_items(all_groups, 8)
print(f"\n--- Example with Budget 8 ---")
print(f"Final State:     {final_solution_8}")

All Groups: [[(5, 10), (4, 8)], [(3, 7)], [(2, 5), (3, 9)]]
Max Budget: 10
---
Highest Benefit: 24
Fewest Items:    3
Final State:     State(benefit=24, count=3)

--- Example with Budget 8 ---
Final State:     State(benefit=19, count=2)


In [ ]:
def solve_game(nums):
    """
    Solves the 'pick two' game using the dynamic programming
    approach from the image.

    Args:
        nums: The list of numbers (A)

    Returns:
        The highest score Yourae can guarantee.
    """

    n = len(nums)
    if n == 0:
        return 0

    # dp[i][j] = max score ADVANTAGE the current player can get
    # from the subarray nums[i...j]
    dp = [[0 for _ in range(n)] for _ in range(n)]

    def get_dp(i, j):
        """
        Helper function to safely get DP values.
        If the subarray is invalid (i > j), it means
        no items are left, so the advantage is 0.
        """
        if i > j:
            return 0
        return dp[i][j]

    # Iterate by the length of the subarray
    # Length must be even: 2, 4, 6, ... n
    for length in range(2, n + 1, 2):

        # Iterate for the start of the subarray
        for i in range(n - length + 1):
            j = i + length - 1  # End of the subarray

            # --- This is the recurrence from your image ---

            # Case 1: Yourae picks A[i], A[i+1]
            # Her gain: A[i] + A[i+1]
            # Ming's advantage on rest: F[i+2][j]
            # Her total advantage: (Gain) - (Opponent's Advantage)
            choice1 = (nums[i] + nums[i + 1]) - get_dp(i + 2, j)

            # Case 2: Yourae picks A[j], A[j-1]
            # Her gain: A[j] + A[j-1]
            # Ming's advantage on rest: F[i][j-2]
            choice2 = (nums[j] + nums[j - 1]) - get_dp(i, j - 2)

            # Case 3: Yourae picks A[i], A[j]
            # Her gain: A[i] + A[j]
            # Ming's advantage on rest: F[i+1][j-1]
            choice3 = (nums[i] + nums[j]) - get_dp(i + 1, j - 1)

            # DP[i][j] is the max of these choices
            dp[i][j] = max(choice1, choice2, choice3)

    # --- Convert Advantage to Final Score ---

    # dp[0][n-1] is Yourae's final advantage
    max_advantage = dp[0][n - 1]

    total_sum = sum(nums)

    # We have two equations:
    # 1) Yourae_Score + Ming_Score = total_sum
    # 2) Yourae_Score - Ming_Score = max_advantage
    #
    # Add them together:
    # 2 * Yourae_Score = total_sum + max_advantage

    yourae_score = (total_sum + max_advantage) / 2

    # Return as an integer since scores are integers
    return int(yourae_score)

# --- Example Usage ---
A = [5, 1, 3, 9, 6, 2, 4, 8]

# Calculate the score for Part 2
# (The image's logic is for Part 2)
score = solve_game(A)

print(f"List: {A}")
print(f"Highest score Yourae can guarantee: {score}")

# Example 2:
A_small = [10, 1, 1, 100]
# Yourae can take (10, 100) -> 110. Ming gets (1, 1) -> 2. Score: 110
# Yourae can take (10, 1) -> 11. Ming gets (1, 100) -> 101. Score: 11
# Yourae can take (1, 100) -> 101. Ming gets (10, 1) -> 11. Score: 101
# Max score is 110.
score_small = solve_game(A_small)
print(f"\nList: {A_small}")
print(f"Highest score Yourae can guarantee: {score_small}")

List: [5, 1, 3, 9, 6, 2, 4, 8]
Highest score Yourae can guarantee: 25

List: [10, 1, 1, 100]
Highest score Yourae can guarantee: 110
